<a href="https://www.kaggle.com/code/aabdollahii/the-art-of-tokenization?scriptVersionId=270740837" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# The Art of Tokenization, Part 1: Byte Pair Encoding

Welcome to this series on the art and science of tokenization! In Natural Language Processing (NLP), before a powerful model like GPT or BERT can understand human language, we must first break that text down into pieces it can recognize. These pieces are called tokens. The process of creating them is tokenization, and it’s one of the most fundamental steps in any NLP pipeline.

This series will explore the different strategies for tokenization. We’ll start with one of the most influential methods in the modern NLP landscape: Byte Pair Encoding (BPE).

# What is BPE? The “Happy Medium” of Tokenization
At its core, Byte Pair Encoding (BPE) is a subword tokenization algorithm. Instead of forcing us to choose between whole words or individual characters, it finds a “happy medium.”


Imagine trying to create a dictionary for a language model:
* Option A: Word Dictionary. You include every single word ("cat", "run", "photosynthesis", "antidisestablishmentarianism").

* Problem: The dictionary becomes enormous. What about new words (“de-platforming”), slang (“yeet”), or simple typos (“helllo”)? They are all “Out-of-Vocabulary” (OOV) and become a meaningless <UNK> (unknown) token.

* Option B: Character Dictionary. You only include characters ('a', 'b', 'c', '!').

* Problem: No more OOV issues, but the text sequences become incredibly long. The sentence “Hello world” is now 11 tokens ('H', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd'). This makes it computationally expensive and harder for the model to find meaning.

This way, a common word like "where" can be a single token, while a rare, complex word like "retrofitting" can be broken down into meaningful pieces like ["retro", "fit", "ting"]. Crucially, no word is ever “unknown.” Any new word can be built from these subword pieces.

# How the BPE Algorithm Works (Conceptually)
BPE was originally a data compression algorithm. Its adaptation for NLP is based on a simple, greedy, and iterative idea:
> Core Logic: Continuously find the most common pair of adjacent tokens in your text and merge them into a single, new token.


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

<h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">How the BPE Algorithm Works (Conceptually)</h3>
    <p style="color: #a0aec0;">BPE is based on a simple, greedy, and iterative idea: continuously find the most common pair of adjacent tokens and merge them. Let's trace the process.</p>

 <style>
        .bpe-kbd-dark {
            background-color: #1a202c;
            border: 1px solid #4a5568;
            border-bottom: 2px solid #718096;
            border-radius: 4px;
            padding: 3px 6px;
            font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
            font-size: 0.9em;
            color: #cbd5e1;
            white-space: nowrap;
        }
    </style>

<div style="margin-top: 25px;">
        <h4 style="color: #a0aec0; margin-bottom: 5px;">Step 0: Initialize</h4>
        <p style="margin: 0;">
            First, we break every word down into its basic characters. We also add a special symbol, like <span class="bpe-kbd-dark">&lt;/w&gt;</span>, to mark the end of a word. This is important to distinguish between <span class="bpe-kbd-dark">er</span> inside a word (like in “newest”) and <span class="bpe-kbd-dark">er</span> at the end of a word (like in “lower”).
        </p>
        <p style="margin-top: 10px;">
            Our initial “tokens” are just characters: <span class="bpe-kbd-dark">l</span> <span class="bpe-kbd-dark">o</span> <span class="bpe-kbd-dark">w</span> <span class="bpe-kbd-dark">e</span> <span class="bpe-kbd-dark">r</span> <span class="bpe-kbd-dark">n</span> <span class="bpe-kbd-dark">s</span> <span class="bpe-kbd-dark">t</span> <span class="bpe-kbd-dark">i</span> <span class="bpe-kbd-dark">d</span> <span class="bpe-kbd-dark">&lt;/w&gt;</span>.
        </p>
    </div>

<div style="margin-top: 25px; border-top: 1px dashed #4a5568; padding-top: 20px;">
        <h4 style="color: #a0aec0; margin-bottom: 5px;">Step 1: First Merge</h4>
        <p>The algorithm scans the entire corpus and counts the frequency of every adjacent pair of tokens.</p>
        <p>Let’s say it finds that the pair <span class="bpe-kbd-dark">e</span> followed by <span class="bpe-kbd-dark">r</span> (i.e., <span class="bpe-kbd-dark">e r</span>) is the most common combination (appearing in <em>lower</em> and <em>wider</em>).</p>
        <ul style="list-style-type: '➡️'; padding-left: 20px; margin-top: 10px;">
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Action:</strong> It merges <span class="bpe-kbd-dark">e</span> and <span class="bpe-kbd-dark">r</span> to create a new token, <span class="bpe-kbd-dark">er</span>.</li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>New Vocabulary:</strong> Our vocabulary now includes <span class="bpe-kbd-dark">er</span>.</li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Corpus Update:</strong> All instances of <span class="bpe-kbd-dark">e r</span> are replaced. So, <span class="bpe-kbd-dark">l o w e r &lt;/w&gt;</span> becomes <span class="bpe-kbd-dark">l o w er &lt;/w&gt;</span>.</li>
        </ul>
    </div>

 <div style="margin-top: 25px; border-top: 1px dashed #4a5568; padding-top: 20px;">
        <h4 style="color: #a0aec0; margin-bottom: 5px;">Step 2: Second Merge</h4>
        <p>The process repeats. The algorithm scans the <em>updated</em> corpus. Perhaps it now finds that <span class="bpe-kbd-dark">er</span> followed by <span class="bpe-kbd-dark">&lt;/w&gt;</span> (i.e., <span class="bpe-kbd-dark">er &lt;/w&gt;</span>) is the most frequent pair.</p>
        <ul style="list-style-type: '➡️'; padding-left: 20px; margin-top: 10px;">
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Action:</strong> It merges <span class="bpe-kbd-dark">er</span> and <span class="bpe-kbd-dark">&lt;/w&gt;</span> to create a new token, <span class="bpe-kbd-dark">er&lt;/w&gt;</span>.</li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>New Vocabulary:</strong> Our vocabulary now includes <span class="bpe-kbd-dark">er</span> and <span class="bpe-kbd-dark">er&lt;/w&gt;</span>.</li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Corpus Update:</strong> <span class="bpe-kbd-dark">l o w er &lt;/w&gt;</span> becomes <span class="bpe-kbd-dark">l o w er&lt;/w&gt;</span>.</li>
        </ul>
    </div>
    
 <div style="margin-top: 25px; border-top: 1px dashed #4a5568; padding-top: 20px;">
        <h4 style="color: #a0aec0; margin-bottom: 5px;">Step 3: Keep Going…</h4>
        <p>Next, maybe <span class="bpe-kbd-dark">l</span> followed by <span class="bpe-kbd-dark">o</span> (<span class="bpe-kbd-dark">l o</span>) is the most common. They get merged into <span class="bpe-kbd-dark">lo</span>. Then <span class="bpe-kbd-dark">lo</span> and <span class="bpe-kbd-dark">w</span> get merged into <span class="bpe-kbd-dark">low</span>.</p>
    </div>

 <div style="margin-top: 25px; background-color: #1a202c; border-left: 5px solid #805ad5; padding: 15px;">
        <h4 style="color: #b794f4; margin: 0 0 5px 0;">The End Result</h4>
        <p style="margin: 0; color: #a0aec0;">This process is repeated for a predetermined number of merges (e.g., <strong>30,000 times</strong>). The final vocabulary consists of the initial characters plus all the new tokens created during the merges. This gives us a ranked list of merge rules that we can use to tokenize any new text.</p>
    </div>
</div>


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

  <h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">
    Pros and Cons of BPE
  </h3>
  <p style="color: #a0aec0;">
    BPE is widely used for a reason, but it’s not without its trade‑offs.
  </p>

  <style>
    .bpe-kbd-dark {
      background-color: #1a202c;
      border: 1px solid #4a5568;
      border-bottom: 2px solid #718096;
      border-radius: 4px;
      padding: 3px 6px;
      font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
      font-size: 0.9em;
      color: #cbd5e1;
      white-space: nowrap;
    }
  </style>

  <!-- ✅ PROS SECTION -->
  <div style="margin-top: 25px;">
    <h4 style="color: #68d391; border-bottom: 1px solid #4a5568; padding-bottom: 5px;">
      Advantages (Pros) 👍
    </h4>
    <ul style="list-style-type: '✔️'; padding-left: 25px; margin-top: 15px;">
      <li style="margin-bottom: 10px;">
        <strong>Eliminates Out‑of‑Vocabulary (OOV) Words:</strong> Any new or rare word can be decomposed into a sequence of known subword tokens. The model never has to deal with a completely 
        <span class="bpe-kbd-dark">&lt;UNK&gt;</span> token.
      </li>
      <li style="margin-bottom: 10px;">
        <strong>Controllable Vocabulary Size:</strong> The final vocabulary size is a hyperparameter — the number of merges. You can balance model size and expressiveness.
      </li>
      <li style="margin-bottom: 10px;">
        <strong>Efficiency:</strong> Common words stay as single tokens, making sequences shorter and inference faster. Rare words are decomposed smartly, making better use of vocab space.
      </li>
      <li style="margin-bottom: 10px;">
        <strong>Captures Morphology:</strong> Learns meaningful parts like prefixes (<span class="bpe-kbd-dark">un-</span>), suffixes (<span class="bpe-kbd-dark">-ing</span>), and stems, helping generalization. If it learns 
        <span class="bpe-kbd-dark">un</span>, it can recognize 
        <em>unhappy</em>, <em>unclear</em>, <em>unbelievable</em>.
      </li>
    </ul>
  </div>

  <!-- ❌ CONS SECTION -->
  <div style="margin-top: 35px; border-top: 1px dashed #4a5568; padding-top: 25px;">
    <h4 style="color: #f56565; border-bottom: 1px solid #4a5568; padding-bottom: 5px;">
      Disadvantages (Cons) 👎
    </h4>
    <ul style="list-style-type: '⚠️'; padding-left: 25px; margin-top: 15px;">
      <li style="margin-bottom: 10px;">
        <strong>Greedy Approach:</strong> BPE makes local (greedy) merges — not necessarily globally optimal. A different merge order might yield a better vocabulary.
      </li>
      <li style="margin-bottom: 10px;">
        <strong>Sensitive to Training Data:</strong> Merge rules depend entirely on the training corpus. A BPE trained on Wikipedia would tokenize Twitter slang or biomedical text poorly.
      </li>
      <li style="margin-bottom: 10px;">
        <strong>The “Byte” Misnomer:</strong> Most modern BPE operates on characters, not raw bytes. True byte‑level BPE (used in GPT‑2) works directly on UTF‑8 bytes and is more robust — handling any text, emoji, or noise.
      </li>
    </ul>
  </div>

  <!-- 🌍 CONTEXT SECTION -->
  <div style="margin-top: 35px; border-top: 1px dashed #4a5568; padding-top: 25px;">
    <h4 style="color: #b794f4; border-bottom: 1px solid #4a5568; padding-bottom: 5px;">
      BPE’s Place in the Tokenization World
    </h4>
    <p style="color: #a0aec0;">
      BPE is a cornerstone method for subword tokenization, but it’s not alone. Its relatives include:
    </p>
    <ul style="list-style-type: '🔹'; padding-left: 25px; margin-top: 15px;">
      <li style="margin-bottom: 10px;">
        <strong>WordPiece</strong> (<span class="bpe-kbd-dark">BERT</span>, <span class="bpe-kbd-dark">DistilBERT</span>): Similar to BPE but merges based on the merge that <em>maximizes the likelihood</em> of the data instead of raw frequency.
      </li>
      <li>
        <strong>Unigram Language Model</strong> (<span class="bpe-kbd-dark">T5</span>, <span class="bpe-kbd-dark">ALBERT</span>, <span class="bpe-kbd-dark">XLNet</span>): A probabilistic approach that prunes subwords contributing least to total corpus probability — can produce multiple valid tokenizations per word (useful as regularization).
      </li>
    </ul>
  </div>

  <!-- ⚡ FINAL NOTE -->
  <div style="margin-top: 30px; background-color: #1a202c; border-left: 5px solid #3182ce; padding: 15px; border-radius: 6px;">
    <p style="margin: 0; color: #a0aec0;">
      Now that we understand the theory, let’s roll up our sleeves and build a 
      <strong style="color:#63b3ed;">BPE tokenizer</strong> from scratch 🔧
    </p>
  </div>
</div>


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

<h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">Step 1: Preparing the Training Corpus</h3>

<style>
        .bpe-kbd-dark {
            background-color: #1a202c;
            border: 1px solid #4a5568;
            border-bottom: 2px solid #718096;
            border-radius: 4px;
            padding: 3px 6px;
            font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
            font-size: 0.9em;
            color: #cbd5e1;
            white-space: nowrap;
        }
    </style>

 <h4 style="color: #a0aec0; margin-bottom: 5px;">What this part does:</h4>
    <p style="color: #a0aec0;">
        Before we can start merging pairs, we need a corpus of text in the right format. The BPE algorithm operates on a vocabulary built from words and their frequencies. So, our first step is to:
    </p>
    <ol style="color: #a0aec0; padding-left: 25px;">
        <li style="padding-left: 10px; margin-bottom: 8px;">Take a raw text corpus.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Split it into individual words and count the frequency of each unique word.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Represent each word as a sequence of characters, separated by spaces, and add a special end-of-word marker, like <span class="bpe-kbd-dark">&lt;/w&gt;</span>.</li>
    </ol>
     <p style="color: #a0aec0; margin-top: 15px;">
        This marker is crucial as it allows the algorithm to distinguish between a subword found inside a larger word (like <span class="bpe-kbd-dark">er</span> in "newer") and a subword that forms the end of a word (like <span class="bpe-kbd-dark">er&lt;/w&gt;</span> in "lower"). Our goal is to transform a block of text into a dictionary where keys are the character-split words and values are their frequencies.
    </p>

 <div style="margin-top: 25px; background-color: #1a202c; border-left: 5px solid #805ad5; padding: 15px;">
        <h4 style="color: #b794f4; margin: 0 0 5px 0;">Example</h4>
         <p style="margin: 0; color: #a0aec0;">
            <strong>Input Corpus:</strong> <code>"low lower newest wider low"</code><br>
            <strong>Becomes:</strong> <code>{'l o w </w>': 2, 'l o w e r </w>': 1, ...}</code>
        </p>
    </div>
</div>


In [1]:
import collections

def get_vocab(corpus: str) -> dict:
    """
    Takes a raw text corpus and creates an initial vocabulary.
    
    The vocabulary maps each word (split into characters and with an end-of-word marker)
    to its frequency in the corpus.

    Args:
        corpus (str): A string containing the training text.

    Returns:
        dict: A dictionary where keys are space-separated characters of words
              (e.g., 'l o w </w>') and values are their counts.
    """
    # Split the corpus into words and count their frequencies
    words = corpus.strip().split()
    word_counts = collections.Counter(words)
    
    # Initialize the vocabulary dictionary
    vocab = {}
    
    # For each word and its count, format it for BPE
    for word, count in word_counts.items():
        # Join characters with a space and add the end-of-word marker
        bpe_word = ' '.join(list(word)) + ' </w>'
        vocab[bpe_word] = count
        
    return vocab

# --- Let's test it with our example corpus ---

corpus_text = "low lower newest wider low low"
initial_vocab = get_vocab(corpus_text)

print("--- Initial Vocabulary ---")
# Use a nicer format for printing the dictionary
for word, count in initial_vocab.items():
    print(f"'{word}': {count}")


--- Initial Vocabulary ---
'l o w </w>': 3
'l o w e r </w>': 1
'n e w e s t </w>': 1
'w i d e r </w>': 1


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

<h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">Step 2: Finding the Most Frequent Pair</h3>

<style>
        .bpe-kbd-dark {
            background-color: #1a202c;
            border: 1px solid #4a5568;
            border-bottom: 2px solid #718096;
            border-radius: 4px;
            padding: 3px 6px;
            font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
            font-size: 0.9em;
            color: #cbd5e1;
            white-space: nowrap;
        }
    </style>

 <h4 style="color: #a0aec0; margin-bottom: 5px;">What this part does:</h4>
    <p style="color: #a0aec0;">
        This is the "counting" step of the BPE loop. At each iteration, we need to scan our entire current vocabulary and count the occurrences of all adjacent pairs of symbols.
    </p>
    <p style="color: #a0aec0; margin-top: 15px;">
        For example, in the vocabulary <code style="color: #e2e8f0;">{'l o w </w>': 3, 'l o w e r </w>': 1}</code>, the pair <span class="bpe-kbd-dark">l o</span> appears in both keys. Its total count would be its count in the first word (3) plus its count in the second word (1), for a total of <strong>4</strong>.
    </p>
     <p style="color: #a0aec0; margin-top: 15px;">
        This function will iterate through every word-tokenization in our vocabulary, find all adjacent pairs within it, and sum up their counts, weighted by the frequency of the word itself. The function will return a dictionary of pairs and their total frequencies.
    </p>
</div>


In [2]:
import collections

# We assume 'initial_vocab' exists from the previous cell.
# If running this cell alone, uncomment the line below:
# initial_vocab = {'l o w </w>': 3, 'l o w e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

def get_pair_stats(vocab: dict) -> collections.Counter:
    """
    Counts the frequency of each adjacent pair of symbols in the vocabulary.

    Args:
        vocab (dict): The current vocabulary mapping token sequences to their counts.

    Returns:
        collections.Counter: A Counter object mapping pairs (tuples) to their frequencies.
    """
    pair_counts = collections.Counter()
    
    for word, count in vocab.items():
        symbols = word.split()
        
        # Iterate through symbols to find adjacent pairs
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i+1])
            # Increment the pair's count by the frequency of the word it appeared in
            pair_counts[pair] += count
            
    return pair_counts

# --- Let's test it with our initial_vocab from Step 1 ---

pair_stats = get_pair_stats(initial_vocab)

print("--- Pair Frequencies (Sorted) ---")
# .most_common() conveniently sorts them from most to least common
for pair, count in pair_stats.most_common():
    print(f"{pair}: {count}")


--- Pair Frequencies (Sorted) ---
('l', 'o'): 4
('o', 'w'): 4
('w', '</w>'): 3
('w', 'e'): 2
('e', 'r'): 2
('r', '</w>'): 2
('n', 'e'): 1
('e', 'w'): 1
('e', 's'): 1
('s', 't'): 1
('t', '</w>'): 1
('w', 'i'): 1
('i', 'd'): 1
('d', 'e'): 1


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

<h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">Step 3: Merging the Best Pair</h3>

<style>
        .bpe-kbd-dark {
            background-color: #1a202c;
            border: 1px solid #4a5568;
            border-bottom: 2px solid #718096;
            border-radius: 4px;
            padding: 3px 6px;
            font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
            font-size: 0.9em;
            color: #cbd5e1;
            white-space: nowrap;
        }
    </style>

 <h4 style="color: #a0aec0; margin-bottom: 5px;">What this part does:</h4>
    <p style="color: #a0aec0;">
        Once we've identified the most frequent pair using our <code>get_pair_stats</code> function, the next logical step is to merge that pair into a single, new token. This is the core "encoding" step.
    </p>
    <p style="color: #a0aec0; margin-top: 15px;">
        This operation involves creating a new vocabulary by iterating through the old one. For each word in the vocabulary, we find all occurrences of the target pair (e.g., <span class="bpe-kbd-dark">l o</span>) and replace them with the newly merged token (e.g., <span class="bpe-kbd-dark">lo</span>). The space between the original pair is removed, signifying they are now one unit.
    </p>

 <div style="margin-top: 25px; background-color: #1a202c; border-left: 5px solid #805ad5; padding: 15px;">
        <h4 style="color: #b794f4; margin: 0 0 5px 0;">Example</h4>
        <ul style="list-style-type: '➡️'; padding-left: 20px; margin-top: 10px; color: #a0aec0;">
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Best Pair to Merge:</strong> <code>('l', 'o')</code></li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>Old Word Tokenization:</strong> <span class="bpe-kbd-dark">l o w </w></span></li>
            <li style="padding-left: 10px; margin-bottom: 8px;"><strong>New Word Tokenization:</strong> <span class="bpe-kbd-dark">lo w </w></span></li>
        </ul>
    </div>
</div>


In [3]:
import re

def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """
    Merges the most frequent pair in the vocabulary.

    Args:
        pair (tuple): The pair of symbols to merge (e.g., ('l', 'o')).
        v_in (dict): The vocabulary before the merge.

    Returns:
        dict: The vocabulary after the merge.
    """
    v_out = {}
    
    # The pair as a string with a space, for regex replacement.
    # We use re.escape to handle tokens that might contain special regex characters.
    bigram = re.escape(' '.join(pair))
    
    # The merged pair as a single token string
    p = ''.join(pair)
    
    # Compile a regex to find the bigram. The (?<!\S) and (?!\S) are negative
    # lookarounds that ensure we are matching the whole token, not a substring of another token.
    # For example, it prevents merging 's' 't' inside 'est'.
    regex = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # Replace the pair with the merged token in the word representation
        w_out = regex.sub(p, word)
        v_out[w_out] = v_in[word]
        
    return v_out

# --- Let's test it with our previous results ---

# From our pair_stats, the best pair is ('l', 'o') with a count of 4.
# (If there's a tie, we just pick one. 'o w' also had a count of 4).
best_pair = ('l', 'o')

# We use 'initial_vocab' from Step 1.
# If running this cell alone, uncomment the line below:
# initial_vocab = {'l o w </w>': 3, 'l o w e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}

# Perform the merge
merged_vocab = merge_vocab(best_pair, initial_vocab)

print(f"--- Vocabulary after merging {best_pair} ---")
for word, count in merged_vocab.items():
    print(f"'{word}': {count}")


--- Vocabulary after merging ('l', 'o') ---
'lo w </w>': 3
'lo w e r </w>': 1
'n e w e s t </w>': 1
'w i d e r </w>': 1


<div style="background-color: #2d3748; color: #e2e8f0; border: 1px solid #4a5568; border-radius: 10px; padding: 25px; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; line-height: 1.7;">

<h3 style="color: #90cdf4; border-bottom: 2px solid #4a5568; padding-bottom: 10px; margin-top: 0;">Step 4: Putting It All Together - The Training Loop</h3>

 <style>
        .bpe-kbd-dark {
            background-color: #1a202c;
            border: 1px solid #4a5568;
            border-bottom: 2px solid #718096;
            border-radius: 4px;
            padding: 3px 6px;
            font-family: 'SFMono-Regular', Consolas, 'Liberation Mono', Menlo, Courier, monospace;
            font-size: 0.9em;
            color: #cbd5e1;
            white-space: nowrap;
        }
    </style>

 <h4 style="color: #a0aec0; margin-bottom: 5px;">What this part does:</h4>
    <p style="color: #a0aec0;">
        This is where we orchestrate the entire BPE training process. We'll create a main function that runs a loop for a specified number of merges (<code>num_merges</code>). This hyperparameter controls the size of our final vocabulary.
    </p>
    <p style="color: #a0aec0; margin-top: 15px;">In each iteration of the loop, the algorithm will perform the following actions:</p>
    <ol style="color: #a0aec0; padding-left: 25px;">
        <li style="padding-left: 10px; margin-bottom: 8px;">Call <code>get_pair_stats</code> on the current vocabulary to count all adjacent pairs.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Identify the most frequent pair from the statistics. If no pairs are left, the training stops early.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Record this "best pair" as a new <strong>merge rule</strong>. The ordered list of these rules is the primary output of our BPE training.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Call <code>merge_vocab</code> to update the vocabulary by merging the best pair into a single token.</li>
        <li style="padding-left: 10px; margin-bottom: 8px;">Repeat this process until the desired number of merges is complete.</li>
    </ol>
    <p style="color: #a0aec0; margin-top: 15px;">
        The two final outputs will be the final state of the vocabulary and, more importantly, the ordered list of <strong>merge rules</strong>. This list is what we need to tokenize new, unseen text.
    </p>
</div>


In [4]:
import collections
import re

# We need to include all our helper functions in this cell to make it self-contained
# and runnable in the Kaggle notebook.

def get_vocab(corpus: str) -> dict:
    """Creates an initial vocabulary from a raw text corpus."""
    words = corpus.strip().split()
    word_counts = collections.Counter(words)
    vocab = {' '.join(list(word)) + ' </w>': count for word, count in word_counts.items()}
    return vocab

def get_pair_stats(vocab: dict) -> collections.Counter:
    """Counts the frequency of each adjacent pair of symbols."""
    pair_counts = collections.Counter()
    for word, count in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pair_counts[(symbols[i], symbols[i+1])] += count
    return pair_counts

def merge_vocab(pair: tuple, v_in: dict) -> dict:
    """Merges a specific pair in the vocabulary."""
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = ''.join(pair)
    regex = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = regex.sub(p, word)
        v_out[w_out] = v_in[word]
    return v_out

# --- The Main BPE Training Function ---

def train_bpe(corpus: str, num_merges: int, verbose: bool = True):
    """
    Trains a BPE model on a corpus.

    Args:
        corpus (str): The training text.
        num_merges (int): The number of merge operations to perform.
        verbose (bool): If True, prints the progress of each merge.

    Returns:
        tuple: A tuple containing:
            - The final vocabulary after all merges.
            - A list of the merge rules in the order they were learned.
    """
    # 1. Initialize vocabulary from the corpus
    vocab = get_vocab(corpus)
    
    # This will store our ordered merge rules
    merge_rules = []
    
    if verbose:
        print("--- Initial Vocabulary ---")
        print(vocab)
        print("-" * 50)
    
    for i in range(num_merges):
        # 2. Get pair statistics from the current vocabulary
        pair_stats = get_pair_stats(vocab)
        
        # If there are no more pairs to merge, stop
        if not pair_stats:
            if verbose: print("No more pairs to merge. Stopping early.")
            break
        
        # 3. Find the most frequent pair
        best_pair = max(pair_stats, key=pair_stats.get)
        merge_rules.append(best_pair)
        
        # 4. Merge the best pair in the vocabulary
        vocab = merge_vocab(best_pair, vocab)
        
        if verbose:
            print(f"Merge {i+1}/{num_merges}: Merged {best_pair} -> {''.join(best_pair)}")
            print(f"  Current Vocab: {list(vocab.keys())}\n")
            
    return vocab, merge_rules

# --- Let's run the full training loop on our corpus ---

corpus_text = "low lower newest wider low low"
num_merges_to_perform = 10

final_vocab, learned_rules = train_bpe(corpus_text, num_merges_to_perform)

print("\n" + "="*25 + " TRAINING COMPLETE " + "="*25)
print("\n--- Final Vocabulary State ---")
for word, count in final_vocab.items():
    print(f"'{word}': {count}")

print("\n--- Learned Merge Rules (in order) ---")
for i, rule in enumerate(learned_rules):
    print(f"{i+1}: {rule}")


--- Initial Vocabulary ---
{'l o w </w>': 3, 'l o w e r </w>': 1, 'n e w e s t </w>': 1, 'w i d e r </w>': 1}
--------------------------------------------------
Merge 1/10: Merged ('l', 'o') -> lo
  Current Vocab: ['lo w </w>', 'lo w e r </w>', 'n e w e s t </w>', 'w i d e r </w>']

Merge 2/10: Merged ('lo', 'w') -> low
  Current Vocab: ['low </w>', 'low e r </w>', 'n e w e s t </w>', 'w i d e r </w>']

Merge 3/10: Merged ('low', '</w>') -> low</w>
  Current Vocab: ['low</w>', 'low e r </w>', 'n e w e s t </w>', 'w i d e r </w>']

Merge 4/10: Merged ('e', 'r') -> er
  Current Vocab: ['low</w>', 'low er </w>', 'n e w e s t </w>', 'w i d er </w>']

Merge 5/10: Merged ('er', '</w>') -> er</w>
  Current Vocab: ['low</w>', 'low er</w>', 'n e w e s t </w>', 'w i d er</w>']

Merge 6/10: Merged ('low', 'er</w>') -> lower</w>
  Current Vocab: ['low</w>', 'lower</w>', 'n e w e s t </w>', 'w i d er</w>']

Merge 7/10: Merged ('n', 'e') -> ne
  Current Vocab: ['low</w>', 'lower</w>', 'ne w e s t </